# TV Script Generation

In this project, I'll generate my own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  I'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network I'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Getting the Data

The data is in `./data/Seinfeld_Scripts.txt`
* As a first step, I'll load in this data and look at some samples. 
* Then, I'll be defining and training an RNN to generate a new script!

In [1]:
# load in data
import helper

data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Exploring the Data

I'll write some code to print out some important stats and view different parts of data to get a better deep understanding.

In [2]:
import numpy as np

view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implementing Pre-processing Functions
The first thing to do to any dataset is pre-processing. for this purpose, I will create the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### First: Lookup Table
To create a word embedding, I first need to transform the words to ids.  In this function, I will create two dictionaries:
- Dictionary to go from the words to an id, I'll call `vocab_to_int`
- Dictionary to go from the id to word, I'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    counts = Counter(text)
    
    # sort counter from most frequent to least frequent
    vocab = sorted(counts,key=counts.get,reverse=True)
    
    # create the dictionaries
    vocab_to_int = {word:i for i,word in enumerate(vocab)}
    int_to_vocab = {i:word for word,i in vocab_to_int.items()}
    
    # return tuple of two dictionaries
    return (vocab_to_int, int_to_vocab)

### Second: Tokenizing Punctuation
I'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Thus, I'll Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||" by creating a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. I also made sure I don't use a value that could be confused as a word; for example, instead of using the value "dash", I used "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punctuation_dict={
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_Mark||',
        '?': '||Question_Mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '\n':'||Return||',
         '-':'||Dash||'
         }
        
    return punctuation_dict

## Pre-processing all the data and saving it

Running the code cell below will pre-process all the data and save it to file.

In [5]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Checkpoint
This is my first checkpoint. The preprocessed data has been saved to disk.

In [6]:
# since I could start the notebook checkpoint from this cell, I'll need to import all  packages and modules again when needed
import helper

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Building the Neural Network
In this section, I'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Checking Access to GPU:

In [7]:
import torch

# check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Training on CPU.')

## Input
Let's start with the preprocessed input data. I'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

I can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader.

### Batching
Below I will implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

I will batch words using the DataLoader, but first I will need to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

In this case, the features are words of size `sequence_length`, and their targets are the single word after this sequence of words in each batch.

In [8]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # get the number of batches
    num_batches = len(words) // batch_size
    
    # 
    batch_words = words[:num_batches*batch_size]
    
    feature_tensors, target_tensors = [], []
    
    n_targets = len(batch_words) - sequence_length
    
    for idx in range(n_targets):
        feature_tensors.append(batch_words[idx:idx+sequence_length])
        target_tensors.append(batch_words[idx+sequence_length])

    data = TensorDataset(torch.from_numpy(np.asarray(feature_tensors)), torch.from_numpy(np.asarray(target_tensors)))
    
    dataloader = DataLoader(data, shuffle=True, batch_size=batch_size)
    
    # return a dataloader
    return dataloader

### Testing the dataloader

Below, I'll make sure the dataloader is working properly by generating some test text data, then define a dataloader using the function I defined above. Then, I'm getting some sample batch of inputs `sample_x` and targets `sample_y` from the dataloader.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  1,   2,   3,   4,   5],
        [ 32,  33,  34,  35,  36],
        [ 33,  34,  35,  36,  37],
        [ 13,  14,  15,  16,  17],
        [  3,   4,   5,   6,   7],
        [  5,   6,   7,   8,   9],
        [ 17,  18,  19,  20,  21],
        [ 37,  38,  39,  40,  41],
        [ 18,  19,  20,  21,  22],
        [ 30,  31,  32,  33,  34]])

torch.Size([10])
tensor([  6,  37,  38,  18,   8,  10,  22,  42,  23,  35])


The dataloader seems to be working well.

---
## Building the Neural Network
Below, I'll determine the structure of the neural network and teh behaviour of the forward function.

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # set class variables
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds)
        lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)
        fc_output = self.fc(lstm_out).view(batch_size, -1, self.output_size)
        nn_output = fc_output[:,-1]

        # return one batch of output word scores and the hidden state
        return nn_output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if(train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
            
        return hidden

### Defining forward and backpropagation

Using the RNN class I implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :param hidden: the hidden state of the lstm
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    
    hidden = tuple([each.data for each in hidden])
    rnn.zero_grad()
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

## Neural Network Training

With the structure of the network completed and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The below function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter.

In [12]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # print loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters


In [13]:
# data parameteres
# sequence Length
sequence_length = 10   #  number of words in a sequence
# batch Size
batch_size = 128

# data loader 
train_loader = batch_data(int_text, sequence_length, batch_size)

In [14]:
# training parameters
# number of Epochs
num_epochs = 30
# learning Rate
learning_rate = 0.001

# model parameters
# vocab size
vocab_size = len(vocab_to_int)
# output size
output_size = vocab_size
# embedding Dimension
embedding_dim = 500
# hidden Dimension
hidden_dim = 256
# number of RNN Layers
n_layers = 2

# show stats for every n number of batches
show_every_n_batches = 2000

### Train
In the next cell, I'll train the neural network on the pre-processed data.

In [15]:
# create the model and move it to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# define loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# train the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# save the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 30 epoch(s)...
Epoch:    1/30    Loss: 4.830820937871933

Epoch:    1/30    Loss: 4.311252894759178

Epoch:    1/30    Loss: 4.155782699465751

Epoch:    2/30    Loss: 3.9773656214183233

Epoch:    2/30    Loss: 3.898522824168205

Epoch:    2/30    Loss: 3.875050127506256

Epoch:    3/30    Loss: 3.7613342759422856

Epoch:    3/30    Loss: 3.711876988530159

Epoch:    3/30    Loss: 3.720200090408325

Epoch:    4/30    Loss: 3.631587271902439

Epoch:    4/30    Loss: 3.5991182202100753

Epoch:    4/30    Loss: 3.6249425432682036

Epoch:    5/30    Loss: 3.534274967291927

Epoch:    5/30    Loss: 3.5218406885862352

Epoch:    5/30    Loss: 3.549092997312546

Epoch:    6/30    Loss: 3.4682529785562397

Epoch:    6/30    Loss: 3.4545662757158278

Epoch:    6/30    Loss: 3.4807250720262526

Epoch:    7/30    Loss: 3.4001917386794025

Epoch:    7/30    Loss: 3.39179683470726

Epoch:    7/30    Loss: 3.4454106447696686

Epoch:    8/30    Loss: 3.354646660487928

Epoch:    8/30   

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


I tried different sets of hyperparameters and choosed the best performing set. The only hyperparameter I didn't change much was the n_layers because 2 layers is usually a good choice for most scenarios and increasing it to 3 might give mixed results. the learning rate could be increased through training momentum or LR scheduling to make the converging faster as it approches a local minima. I also think training it for more epochs would have been beneficial.

---
# Checkpoint
This is my second checkpoint. as the model has been saved to the disk.

In [16]:

import torch
import helper

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, I'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. I'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [17]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
Now it's time to generate the text.

As this is a TV script, it's best to start with a name for generating a TV script.
Main charachters in the original scipt:
- "Jerry"
- "Elaine"
- "George"
- "Kramer"

Here, I've choosen `Jerry` as the starting name of the generated script. recall that all words fed to the neural network are lowercase letters that's why I choose `jerry` not `Jerry`.


In [18]:
gen_length = 400 # length of generated words
prime_word = 'jerry' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: invest.

george: no.

jerry: well you didn't have a girlfriend, i am loving it. i'm not going to be a big fan.

kramer: yeah.

george:(to jerry) oh, yeah. i was wondering if i could get some coffee, and you know, i think i may be a little bit.. i don't know what the name is.

jerry:(to the intercom) hello? hello!(to george) you know, i have a very good to the general manager.

george:(entering) oh, ho, ho. ahhhhh!(laughs)

jerry: so, you guys?

elaine: yeah, i got a new plates, though.

kramer: oh yeah?

jerry:(pointing to her) hey!

kramer: hey, you know what happens, buddy???

elaine: what?...

george: i don't know, jerry.

jerry:(cont'd) no, no. i didn't say anything.

helen:(to helen) you can't go to mendys. you want to get a bite, i can't sleep.

jerry:(to the phone) hello?

elaine: hi, jerry.

jerry: hello, jerry.

george: hi.(to george) hey.

george:(to jerry) i think you're right.

jerry: hey.

elaine: hi jerry.

jerry: hi.

george: what are you doing?

jerry: i was hopi

# The TV Script is Not Perfect
It's ok that the TV script doesn't make perfect sense. It looks like alternating lines of dialogue.


you can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, I'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes.


#### Saving scripts

Below, I added some code that saves the generated script to a text file.

In [19]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()